In [ ]:
"""
Purpose: To debug why certain somas are not being detected
in agreement with nucleus table


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import soma_extraction_utils as sm
import datajoint_utils as du
import neuron_visualizations as nviz

INFO - 2021-01-10 02:03:50,623 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-10 02:03:50,625 - settings - Setting database.user to celiib
INFO - 2021-01-10 02:03:50,626 - settings - Setting database.password to newceliipass
INFO - 2021-01-10 02:03:51,112 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-10 02:03:51,113 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-10 02:03:51,131 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-10 02:03:51,415 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-10 02:03:51,511 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-10 02:03:51,794 - settings - Setting enable_python_native_blobs to True


# Checking what neurons did not have 2 somas which should have (so can test on them now)

# 1/6 Debugging Missed Soma

In [7]:
segment_id = 864691134947393276

In [8]:
current_neuron = du.fetch_segment_id_mesh(segment_id)

In [9]:
nviz.plot_objects(current_neuron)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# The function that is unpacked

In [18]:
big_splits = tu.split_significant_pieces(recov_orig_mesh_no_interior,5000,connectivity="edges")
big_splits

[<trimesh.Trimesh(vertices.shape=(586881, 3), faces.shape=(1169394, 3))>,
 <trimesh.Trimesh(vertices.shape=(59680, 3), faces.shape=(120366, 3))>,
 <trimesh.Trimesh(vertices.shape=(50257, 3), faces.shape=(101037, 3))>,
 <trimesh.Trimesh(vertices.shape=(34913, 3), faces.shape=(69067, 3))>,
 <trimesh.Trimesh(vertices.shape=(24698, 3), faces.shape=(50303, 3))>,
 <trimesh.Trimesh(vertices.shape=(23757, 3), faces.shape=(47575, 3))>,
 <trimesh.Trimesh(vertices.shape=(18843, 3), faces.shape=(37026, 3))>,
 <trimesh.Trimesh(vertices.shape=(14417, 3), faces.shape=(29025, 3))>,
 <trimesh.Trimesh(vertices.shape=(14157, 3), faces.shape=(28316, 3))>,
 <trimesh.Trimesh(vertices.shape=(13478, 3), faces.shape=(27359, 3))>,
 <trimesh.Trimesh(vertices.shape=(8588, 3), faces.shape=(17089, 3))>,
 <trimesh.Trimesh(vertices.shape=(4502, 3), faces.shape=(9069, 3))>]

In [20]:
nviz.plot_objects(big_splits[2])

In [26]:
nviz.plot_objects(recov_orig_mesh_no_interior,
                 meshes=glia_pieces,
                 meshes_colors="red")

In [13]:
glia_pieces

[<trimesh.Trimesh(vertices.shape=(524947, 3), faces.shape=(1075745, 3))>]

In [14]:
nviz.plot_objects(meshes=nuclei_pieces)

In [ ]:
glia_pieces, nuclei_pieces

In [30]:
from soma_extraction_utils import *
import numpy as np


segment_id = segment_id
current_mesh_verts = current_neuron.vertices
current_mesh_faces = current_neuron.faces
current_mesh = None


outer_decimation_ratio= 0.25
large_mesh_threshold = 20000#60000,
large_mesh_threshold_inner = 13000 #was changed so dont filter away som somas
soma_width_threshold = 0.32
soma_size_threshold = 9000 #changed this to smaller so didn't filter some somas away
inner_decimation_ratio = 0.25
volume_mulitplier=8
#side_length_ratio_threshold=3
side_length_ratio_threshold=6
soma_size_threshold_max=240000#192000 #this puts at 12000 once decimated, another possible is 256000
delete_files=True
backtrack_soma_mesh_to_original=True #should either be None or 
boundary_vertices_threshold=None#700 the previous threshold used
poisson_backtrack_distance_threshold=None#1500 the previous threshold used
close_holes=False

#------- 11/12 Additions --------------- #

#these arguments are for removing inside pieces
remove_inside_pieces = True
size_threshold_to_remove=1000 #size accounting for the decimation


pymeshfix_clean=False
check_holes_before_pymeshfix=False
second_poisson=False
segmentation_at_end=True
last_size_threshold = 2000#1300,

largest_hole_threshold = 17000
max_fail_loops = 10
perform_pairing = False
verbose = True
return_glia_nuclei_pieces = True

backtrack_soma_size_threshold = 4000


global_start_time = time.time()

#Adjusting the thresholds based on the decimations
large_mesh_threshold = large_mesh_threshold*outer_decimation_ratio
large_mesh_threshold_inner = large_mesh_threshold_inner*outer_decimation_ratio
soma_size_threshold = soma_size_threshold*outer_decimation_ratio
soma_size_threshold_max = soma_size_threshold_max*outer_decimation_ratio

#adjusting for inner decimation
soma_size_threshold = soma_size_threshold*inner_decimation_ratio
soma_size_threshold_max = soma_size_threshold_max*inner_decimation_ratio
print(f"Current Arguments Using (adjusted for decimation):\n large_mesh_threshold= {large_mesh_threshold}"
             f" \nlarge_mesh_threshold_inner = {large_mesh_threshold_inner}"
              f" \nsoma_size_threshold = {soma_size_threshold}"
             f" \nsoma_size_threshold_max = {soma_size_threshold_max}"
             f"\nouter_decimation_ratio = {outer_decimation_ratio}"
             f"\ninner_decimation_ratio = {inner_decimation_ratio}")


# ------------------------------


temp_folder = f"./{segment_id}"
temp_object = Path(temp_folder)
#make the temp folder if it doesn't exist
temp_object.mkdir(parents=True,exist_ok=True)

#making the decimation and poisson objections
Dec_outer = meshlab.Decimator(outer_decimation_ratio,temp_folder,overwrite=True)
Dec_inner = meshlab.Decimator(inner_decimation_ratio,temp_folder,overwrite=True)
Poisson_obj = meshlab.Poisson(temp_folder,overwrite=True)


recov_orig_mesh = trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)

recov_orig_mesh_no_interior, glia_pieces, nuclei_pieces  = tu.remove_nuclei_and_glia_meshes(recov_orig_mesh,verbose=True)
#recov_orig_mesh_no_interior = tu.remove_mesh_interior(recov_orig_mesh)


#Step 1: Decimate the Mesh and then split into the seperate pieces
new_mesh,output_obj = Dec_outer(vertices=recov_orig_mesh_no_interior.vertices,
         faces=recov_orig_mesh_no_interior.faces,
         segment_id=segment_id,
         return_mesh=True,
         delete_temp_files=False)

# if remove_inside_pieces:
#     print("removing mesh interior after decimation")
#     new_mesh = tu.remove_mesh_interior(new_mesh,size_threshold_to_remove=size_threshold_to_remove)

#preforming the splits of the decimated mesh

mesh_splits = new_mesh.split(only_watertight=False)

#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])


total_mesh_split_lengths = [len(k.faces) for k in mesh_splits]
ordered_mesh_splits = mesh_splits[np.flip(np.argsort(total_mesh_split_lengths))]
list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold]

print(f"Total found significant pieces before Poisson = {list_of_largest_mesh}")

#if no significant pieces were found then will use smaller threshold
if len(list_of_largest_mesh)<=0:
    print(f"Using smaller large_mesh_threshold because no significant pieces found with {large_mesh_threshold}")
    list_of_largest_mesh = [k for k in ordered_mesh_splits if len(k.faces) > large_mesh_threshold/2]

total_soma_list = []
total_classifier_list = []
total_poisson_list = []
total_soma_list_sdf = []



#start iterating through where go through all pieces before the poisson reconstruction
no_somas_found_in_big_loop = 0
for i,largest_mesh in enumerate(list_of_largest_mesh):
    print(f"----- working on large mesh #{i}: {largest_mesh}")

    if remove_inside_pieces:
        print("remove_inside_pieces requested ")
        largest_mesh = tu.remove_mesh_interior(largest_mesh,size_threshold_to_remove=size_threshold_to_remove)


    if pymeshfix_clean:
        print("Requested pymeshfix_clean")
        """
        Don't have to check if manifold anymore actually just have to plug the holes
        """
        hole_groups = tu.find_border_face_groups(largest_mesh)
        if len(hole_groups) > 0:
            largest_mesh_filled_holes = tu.fill_holes(largest_mesh,max_hole_size = 10000)
        else:
            largest_mesh_filled_holes = largest_mesh

        if check_holes_before_pymeshfix:
            hole_groups = tu.find_border_face_groups(largest_mesh_filled_holes)
        else:
            print("Not checking if there are still existing holes before pymeshfix")
            hole_groups = []

        if len(hole_groups) > 0:
            #segmentation_at_end = False
            print(f"*** COULD NOT FILL HOLES WITH MAX SIZE OF {np.max([len(k) for k in hole_groups])} so not applying pymeshfix and segmentation_at_end = {segmentation_at_end}")

#                 tu.write_neuron_off(largest_mesh_filled_holes,"largest_mesh_filled_holes")
#                 raise Exception()
        else:
            print("Applying pymeshfix_clean because no more holes")
            largest_mesh = tu.pymeshfix_clean(largest_mesh_filled_holes,verbose=True)

    if second_poisson:
        print("Applying second poisson run")
        current_neuron_poisson = tu.poisson_surface_reconstruction(largest_mesh)
        largest_mesh = tu.split_significant_pieces(current_neuron_poisson)[0]

    somas_found_in_big_loop = False

    largest_file_name = str(output_obj.stem) + "_largest_piece.off"
    pre_largest_mesh_path = temp_object / Path(str(output_obj.stem) + "_largest_piece.off")
    pre_largest_mesh_path = pre_largest_mesh_path.absolute()
    print(f"pre_largest_mesh_path = {pre_largest_mesh_path}")
    # ******* This ERRORED AND CALLED OUR NERUON NONE: 77697401493989254 *********
    new_mesh_inner,poisson_file_obj = Poisson_obj(vertices=largest_mesh.vertices,
               faces=largest_mesh.faces,
               return_mesh=True,
               mesh_filename=largest_file_name,
               delete_temp_files=False)


    #splitting the Poisson into the largest pieces and ordering them
    mesh_splits_inner = new_mesh_inner.split(only_watertight=False)
    total_mesh_split_lengths_inner = [len(k.faces) for k in mesh_splits_inner]
    ordered_mesh_splits_inner = mesh_splits_inner[np.flip(np.argsort(total_mesh_split_lengths_inner))]

    list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]
    print(f"Total found significant pieces AFTER Poisson = {list_of_largest_mesh_inner}")

    n_failed_inner_soma_loops = 0
    for j, largest_mesh_inner in enumerate(list_of_largest_mesh_inner):
        to_add_list = []
        to_add_list_sdf = []

        print(f"----- working on mesh after poisson #{j}: {largest_mesh_inner}")

        largest_mesh_path_inner = str(poisson_file_obj.stem) + "_largest_inner.off"

        #Decimate the inner poisson piece
        largest_mesh_path_inner_decimated,output_obj_inner = Dec_inner(
                            vertices=largest_mesh_inner.vertices,
                             faces=largest_mesh_inner.faces,
                            mesh_filename=largest_mesh_path_inner,
                             return_mesh=True,
                             delete_temp_files=False)

        dec_splits = tu.split_significant_pieces(largest_mesh_path_inner_decimated,significance_threshold=15)
        print(f"\n-------Splits after inner decimation len = {len(dec_splits)}--------\n")

        if len(dec_splits) == 0:
            print("There were no signifcant splits after inner decimation")
            n_failed_inner_soma_loops += 1
        else:
            print(f"done exporting decimated mesh: {largest_mesh_path_inner}")

            largest_mesh_path_inner_decimated_clean = dec_splits[0]
            print(f"largest_mesh_path_inner_decimated_clean = {largest_mesh_path_inner_decimated_clean}\n")

            faces = np.array(largest_mesh_path_inner_decimated_clean.faces)
            verts = np.array(largest_mesh_path_inner_decimated_clean.vertices)

            # may need to do some processing


            segment_id_new = int(str(segment_id) + f"{i}{j}")
            #print(f"Before the classifier the pymeshfix_clean = {pymeshfix_clean}")
            verts_labels, faces_labels, soma_value,classifier = wcda.extract_branches_whole_neuron(
                                    import_Off_Flag=False,
                                    segment_id=segment_id_new,
                                    vertices=verts,
                                     triangles=faces,
                                    pymeshfix_Flag=False,
                                     import_CGAL_Flag=False,
                                     return_Only_Labels=True,
                                     clusters=3,
                                     smoothness=0.2,
                                    soma_only=True,
                                    return_classifier = True
                                    )
            print(f"soma_sdf_value = {soma_value}")

            total_classifier_list.append(classifier)
            #total_poisson_list.append(largest_mesh_path_inner_decimated)

            # Save all of the portions that resemble a soma
            median_values = np.array([v["median"] for k,v in classifier.sdf_final_dict.items()])
            segmentation = np.array([k for k,v in classifier.sdf_final_dict.items()])

            #order the compartments by greatest to smallest
            sorted_medians = np.flip(np.argsort(median_values))
            print(f"segmentation[sorted_medians],median_values[sorted_medians] = {(segmentation[sorted_medians],median_values[sorted_medians])}")
            print(f"Sizes = {[classifier.sdf_final_dict[g]['n_faces'] for g in segmentation[sorted_medians]]}")
            print(f"soma_size_threshold = {soma_size_threshold}")
            print(f"soma_size_threshold_max={soma_size_threshold_max}")

            valid_soma_segments_width = [g for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]
            valid_soma_segments_sdf = [h for g,h in zip(segmentation[sorted_medians],median_values[sorted_medians]) if ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))]

            print("valid_soma_segments_width")

            if len(valid_soma_segments_width) > 0:
                print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
                somas_found_in_big_loop = True
                #get the meshes only if signfiicant length
                labels_list = classifier.labels_list

                for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
                    submesh_face_list = np.where(classifier.labels_list == v)[0]
                    soma_mesh = largest_mesh_path_inner_decimated_clean.submesh([submesh_face_list],append=True)

                    # ---------- No longer doing the extra checks in here --------- #


                    curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
                    curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)


                    if curr_side_len_check and curr_volume_check:
                        #check if we can split this into two

                        possible_smoothness = [0.2,0.05,0.01]
                        for smooth_value in possible_smoothness:
                            #1) Run th esegmentation algorithm again to segment the mesh (had to run the higher smoothing to seperate some)
                            mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(soma_mesh,clusters=3,smoothness=smooth_value,verbose=True)
                            mesh_extra = np.array(mesh_extra)

                            #2) Filter out meshes by sizs and sdf threshold
                            mesh_extra_lens = np.array([len(kk.faces) for kk in mesh_extra])
                            filtered_meshes_idx = np.where((mesh_extra_lens >= soma_size_threshold) & (mesh_extra_lens <= soma_size_threshold_max) & (mesh_extra_sdf>soma_width_threshold))[0]

                            if len(filtered_meshes_idx) >= 2:
                                if verbose:
                                    print(f"Breakin on smoothness: {smooth_value}")
                                break

                        if len(filtered_meshes_idx) >= 2:
                            filtered_meshes = mesh_extra[filtered_meshes_idx]
                            filtered_meshes_sdf = mesh_extra_sdf[filtered_meshes_idx]

                            to_add_list_retry = []
                            to_add_list_sdf_retry = []

                            for f_m,f_m_sdf in zip(filtered_meshes,filtered_meshes_sdf):
                                curr_side_len_check_retry = side_length_check(f_m,side_length_ratio_threshold)
                                curr_volume_check_retry = soma_volume_check(f_m,volume_mulitplier)

                                if curr_side_len_check_retry and curr_volume_check_retry:
                                    to_add_list_retry.append(f_m)
                                    to_add_list_sdf_retry.append(f_m_sdf)

                            if len(to_add_list_retry)>1:
                                if verbose:
                                    print("Using the new feature that split the soma further into more groups")
                                to_add_list += to_add_list_retry
                                to_add_list_sdf += to_add_list_sdf_retry

                            else:
                                to_add_list.append(soma_mesh)
                                to_add_list_sdf.append(sdf)


                        else:
                            to_add_list.append(soma_mesh)
                            to_add_list_sdf.append(sdf)

                    else:
                        # ---------- 1/7 Addition: Trying one more additional cgal segmentation to see if there is actually a soma ---
                        """
                        Pseudocode: 
                        1) Run th esegmentation algorithm again to segment the mesh
                        2) Filter out meshes by sizs and sdf threshold
                        3) If there are any remaining meshes, pick the largest sdf mesh and test for volume and side length check
                        --> if matches then adds
                        """

                        print(f"->Attempting retry of soma because failed first checks: "
                                 f"soma_mesh = {soma_mesh}, curr_side_len_check = {curr_side_len_check}, curr_volume_check = {curr_volume_check}")
                        #1) Run th esegmentation algorithm again to segment the mesh
                        mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(soma_mesh,clusters=3,smoothness=0.2,verbose=True)
                        mesh_extra = np.array(mesh_extra)

                        #2) Filter out meshes by sizs and sdf threshold
                        mesh_extra_lens = np.array([len(kk.faces) for kk in mesh_extra])
                        filtered_meshes_idx = np.where((mesh_extra_lens >= soma_size_threshold) & (mesh_extra_lens <= soma_size_threshold_max) & (mesh_extra_sdf>soma_width_threshold))[0]


                        if len(filtered_meshes_idx) > 0:
                            filtered_meshes = mesh_extra[filtered_meshes_idx]
                            filtered_meshes_sdf = mesh_extra_sdf[filtered_meshes_idx]

                            sdf_winning_index = np.argmax(filtered_meshes_sdf)
                            soma_mesh_retry = filtered_meshes[sdf_winning_index]
                            sdf_retry = filtered_meshes_sdf[sdf_winning_index]

                            curr_side_len_check_retry = side_length_check(soma_mesh_retry,side_length_ratio_threshold)
                            curr_volume_check_retry = soma_volume_check(soma_mesh_retry,volume_mulitplier)

                            if curr_side_len_check_retry and curr_volume_check_retry:
                                to_add_list.append(soma_mesh_retry)
                                to_add_list_sdf.append(sdf_retry)
                            else:
                                print(f"--->This soma mesh was not added because failed retry of sphere validation:\n "
                                     f"soma_mesh = {soma_mesh_retry}, curr_side_len_check = {curr_side_len_check_retry}, curr_volume_check = {curr_volume_check_retry}")
                                continue
                        else:
                            print(f"Could not find valid soma mesh in retry")
                            continue


                n_failed_inner_soma_loops = 0

            else:
                n_failed_inner_soma_loops += 1

        total_soma_list_sdf += to_add_list_sdf
        total_soma_list += to_add_list

        # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
        if n_failed_inner_soma_loops >= max_fail_loops:
            print(f"breaking inner loop because {max_fail_loops} soma fails in a row")
            break


    # --------------- KEEP TRACK IF FAILED TO FIND SOMA (IF TOO MANY FAILS THEN BREAK)
    if somas_found_in_big_loop == False:
        no_somas_found_in_big_loop += 1
        if no_somas_found_in_big_loop >= max_fail_loops:
            print(f"breaking because {max_fail_loops} fails in a row in big loop")
            break

    else:
        no_somas_found_in_big_loop = 0





""" IF THERE ARE MULTIPLE SOMAS THAT ARE WITHIN A CERTAIN DISTANCE OF EACH OTHER THEN JUST COMBINE THEM INTO ONE"""
pairings = []

if perform_pairing:
    for y,soma_1 in enumerate(total_soma_list):
        for z,soma_2 in enumerate(total_soma_list):
            if y<z:
                mesh_tree = KDTree(soma_1.vertices)
                distances,closest_node = mesh_tree.query(soma_2.vertices)

                if np.min(distances) < 4000:
                    pairings.append([y,z])


#creating the combined meshes from the list
total_soma_list_revised = []
total_soma_list_revised_sdf = []
if len(pairings) > 0:
    """
    Pseudocode: 
    Use a network function to find components

    """


    import networkx as nx
    new_graph = nx.Graph()
    new_graph.add_edges_from(pairings)
    grouped_somas = list(nx.connected_components(new_graph))

    somas_being_combined = []
    print(f"There were soma pairings: Connected components in = {grouped_somas} ")
    for comp in grouped_somas:
        comp = list(comp)
        somas_being_combined += list(comp)
        current_mesh = total_soma_list[comp[0]]
        for i in range(1,len(comp)):
            current_mesh += total_soma_list[comp[i]] #just combining the actual meshes

        total_soma_list_revised.append(current_mesh)
        #where can average all of the sdf values
        total_soma_list_revised_sdf.append(np.min(np.array(total_soma_list_sdf)[comp]))

    #add those that weren't combined to total_soma_list_revised
    leftover_somas = [total_soma_list[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
    leftover_somas_sdfs = [total_soma_list_sdf[k] for k in range(0,len(total_soma_list)) if k not in somas_being_combined]
    if len(leftover_somas) > 0:
        total_soma_list_revised += leftover_somas
        total_soma_list_revised_sdf += leftover_somas_sdfs

    print(f"Final total_soma_list_revised = {total_soma_list_revised}")
    print(f"Final total_soma_list_revised_sdf = {total_soma_list_revised_sdf}")


if len(total_soma_list_revised) == 0:
    total_soma_list_revised = total_soma_list
    total_soma_list_revised_sdf = total_soma_list_sdf

run_time = time.time() - global_start_time

print(f"\n\n\n Total time for run = {time.time() - global_start_time}")
print(f"Before Filtering the number of somas found = {len(total_soma_list_revised)}")

#     import system_utils as su
#     su.compressed_pickle(total_soma_list_revised,"total_soma_list_revised")
#     su.compressed_pickle(new_mesh,"original_mesh")

#need to erase all of the temporary files ******
#import shutil
#shutil.rmtree(directory)

"""
Running the extra tests that depend on
- border vertices
- how well the poisson matches the backtracked soma to the real mesh
- other size checks

"""
filtered_soma_list = []
filtered_soma_list_sdf = []
for soma_mesh,curr_soma_sdf in zip(total_soma_list_revised,total_soma_list_revised_sdf):
    if backtrack_soma_mesh_to_original:
        print("Performing Soma Mesh Backtracking to original mesh")
        soma_mesh_poisson = deepcopy(soma_mesh)
        try:
            #print("About to find original mesh")
            soma_mesh = original_mesh_soma(
                                            mesh = recov_orig_mesh_no_interior,
                                            soma_meshes=[soma_mesh_poisson],
                                            sig_th_initial_split=15)[0]
        except:
            import traceback
            traceback.print_exc()
            print("--->This soma mesh was not added because Was not able to backtrack soma to mesh")
            continue
        else:
            if soma_mesh is None:
                print("--->This soma mesh was not added because Was not able to backtrack soma to mesh")
                continue     

        print(f"poisson_backtrack_distance_threshold = {poisson_backtrack_distance_threshold}")
        #do the check that tests if there is a max distance between poisson and backtrack:
        if not poisson_backtrack_distance_threshold is None and poisson_backtrack_distance_threshold > 0:

            #soma_mesh.export("soma_mesh.off")
            if close_holes: 
                print("Using the close holes feature")
                fill_hole_obj = meshlab.FillHoles(max_hole_size=2000,
                                                 self_itersect_faces=False)

                soma_mesh_filled_holes,output_subprocess_obj = fill_hole_obj(   
                                                    vertices=soma_mesh.vertices,
                                                     faces=soma_mesh.faces,
                                                     return_mesh=True,
                                                     delete_temp_files=True,
                                                    )
            else:
                soma_mesh_filled_holes = soma_mesh


            #soma_mesh_filled_holes.export("soma_mesh_filled_holes.off")



            print("APPLYING poisson_backtrack_distance_threshold CHECKS")
            mesh_1 = soma_mesh_filled_holes
            mesh_2 = soma_mesh_poisson

            poisson_max_distance = tu.max_distance_betwee_mesh_vertices(mesh_1,mesh_2,
                                                              verbose=True)
            print(f"poisson_max_distance = {poisson_max_distance}")
            if poisson_max_distance > poisson_backtrack_distance_threshold:
                print(f"--->This soma mesh was not added because it did not pass the poisson_backtrack_distance check:\n"
                  f" poisson_max_distance = {poisson_max_distance}")
                continue

    if len(soma_mesh.faces) < 5:
        print(f"--> soma had very few faces ({soma_mesh}) so continuing")
        continue

    #do the boundary check:
    if not boundary_vertices_threshold is None:
        print("USING boundary_vertices_threshold CHECK")
        soma_boundary_groups_sizes = np.array([len(k) for k in tu.find_border_face_groups(soma_mesh)])
        print(f"soma_boundary_groups_sizes = {soma_boundary_groups_sizes}")
        large_boundary_groups = soma_boundary_groups_sizes[soma_boundary_groups_sizes>boundary_vertices_threshold]
        print(f"large_boundary_groups = {large_boundary_groups} with boundary_vertices_threshold = {boundary_vertices_threshold}")
        if len(large_boundary_groups)>0:
            print(f"--->This soma mesh was not added because it did not pass the boundary vertices validation:\n"
                  f" large_boundary_groups = {large_boundary_groups}")
            continue

            
    
    curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
    curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)
    if (not curr_side_len_check) or (not curr_volume_check):
        print(f"--->This soma mesh was not added because it did not pass the sphere validation:\n "
             f"soma_mesh = {soma_mesh}, curr_side_len_check = {curr_side_len_check}, curr_volume_check = {curr_volume_check}")
        continue

    #tu.write_neuron_off(soma_mesh_poisson,"original_poisson.off")
    #If made it through all the checks then add to final list
    filtered_soma_list.append(soma_mesh)
    filtered_soma_list_sdf.append(curr_soma_sdf)


"""
Need to delete all files in the temp folder *****
"""

if delete_files:
    #now erase all of the files used
    from shutil import rmtree

    #remove the directory with the meshes
    rmtree(str(temp_object.absolute()))

    #removing the temporary files
    temp_folder = Path("./temp")
    temp_files = [x for x in temp_folder.glob('**/*')]
    seg_temp_files = [x for x in temp_files if str(segment_id) in str(x)]

    for f in seg_temp_files:
        f.unlink()

# ----------- 11 /11 Addition that does a last step segmentation of the soma --------- #
#return total_soma_list, run_time
#return total_soma_list_revised,run_time,total_soma_list_revised_sdf

"""
Things we should ask about the segmentation:

Advantages: 
1) could help filter away negatives

Disadvantages:
1) Can actually cut up the soma and then filter away the soma (not what we want)
2) Could introduce a big hole (don't think can guard against this)
"""


#filtered_soma_list_saved = copy.deepcopy(filtered_soma_list)
import system_utils as su
su.compressed_pickle(filtered_soma_list,"filtered_soma_list")

if len(filtered_soma_list) > 0:
    filtered_soma_list_revised = []
    filtered_soma_list_sdf_revised = []
    for f_soma,f_soma_sdf in zip(filtered_soma_list,filtered_soma_list_sdf):

        print("Skipping the segmentatio filter at end")
        if not (len(f_soma.faces) >= last_size_threshold and f_soma_sdf >= soma_width_threshold):
            print(f"Soma (size = {len(f_soma.faces)}, width={soma_width_threshold}) did not pass thresholds (size threshold={last_size_threshold}, width threshold = {soma_width_threshold}) ")
            continue


        if segmentation_at_end:


            if remove_inside_pieces:
                print("removing mesh interior before segmentation")
                f_soma = tu.remove_mesh_interior(f_soma,size_threshold_to_remove=size_threshold_to_remove)

            print("Doing the soma segmentation filter at end")

            meshes_split,meshes_split_sdf = tu.mesh_segmentation(
                mesh = f_soma,
                smoothness=0.5
            )
#                 print(f"meshes_split = {meshes_split}")
#                 print(f"meshes_split_sdf = {meshes_split_sdf}")

            #applying the soma width and the soma size threshold
            above_width_threshold_mask = meshes_split_sdf>=soma_width_threshold
            meshes_split_sizes = np.array([len(k.faces) for k in meshes_split])
            above_size_threshold_mask = meshes_split_sizes >= last_size_threshold

            above_width_threshold_idx = np.where(above_width_threshold_mask & above_size_threshold_mask)[0]
            if len(above_width_threshold_idx) == 0:
                print(f"No split meshes were above the width threshold ({soma_width_threshold}) and size threshold ({last_size_threshold}) so continuing")
                print(f"So just going with old somas")

                f_soma_final = f_soma
                f_soma_sdf_final = f_soma_sdf


            else:
                meshes_split = np.array(meshes_split)
                meshes_split_sdf = np.array(meshes_split_sdf)

                meshes_split_filtered = meshes_split[above_width_threshold_idx]
                meshes_split_sdf_filtered = meshes_split_sdf[above_width_threshold_idx]

                soma_width_threshold
                #way to choose the index of the top candidate
                top_candidate = 0


                largest_hole_before_seg = tu.largest_hole_length(f_soma)
                largest_hole_after_seg = tu.largest_hole_length(meshes_split_filtered[top_candidate])

                print(f"Largest hole before segmentation = {largest_hole_before_seg}, after = {largest_hole_after_seg},"
                      f"\nratio = {largest_hole_after_seg/largest_hole_before_seg}, difference = {largest_hole_after_seg - largest_hole_before_seg}")

                if largest_hole_after_seg < largest_hole_threshold:
                    f_soma_final = meshes_split_filtered[top_candidate]
                    f_soma_sdf_final = meshes_split_sdf_filtered[top_candidate]
                else:
                    f_soma_final = f_soma
                    f_soma_sdf_final = f_soma_sdf

        else:
            f_soma_final = f_soma
            f_soma_sdf_final = f_soma_sdf


        filtered_soma_list_revised.append(f_soma_final)
        filtered_soma_list_sdf_revised.append(f_soma_sdf_final)




    filtered_soma_list = np.array(filtered_soma_list_revised)
    filtered_soma_list_sdf = np.array(filtered_soma_list_sdf_revised)

    """
    # ----------- 1/7/21 ---------------#
    Now was to stitch the somas together if they are touching

    """
    connected_meshes_components = tu.mesh_list_connectivity(meshes=filtered_soma_list,
                             main_mesh=recov_orig_mesh_no_interior,
                                                return_connected_components=True)

    filtered_soma_list_components = np.array([tu.combine_meshes(filtered_soma_list[k]) for k in connected_meshes_components])
    filtered_soma_list_sdf_components = np.array([np.mean(filtered_soma_list_sdf[k]) for k in connected_meshes_components])
else:
    filtered_soma_list_components = []
    filtered_soma_list_sdf_components = np.array([])
    
    
    
#----------- 1/9 Final Size Threshold ------------- #
if backtrack_soma_mesh_to_original:
    
    filtered_soma_list_components_new = []
    filtered_soma_list_sdf_components_new = []
    
    for soma_mesh, soma_mesh_sdf in zip(filtered_soma_list_components,filtered_soma_list_sdf_components):
        # --------- 1/9: Extra Size Threshold For Somas ------------- #
        if len(soma_mesh.faces) < backtrack_soma_size_threshold:
            print(f"--->This soma mesh with size {len(soma_mesh.faces)} was not bigger than the threshold {backtrack_soma_size_threshold}")
        else:
            filtered_soma_list_components_new.append(soma_mesh)
            filtered_soma_list_sdf_components_new.append(soma_mesh_sdf)
            
    filtered_soma_list_components = filtered_soma_list_components_new
    filtered_soma_list_sdf_components = np.array(filtered_soma_list_sdf_components_new)
            

if return_glia_nuclei_pieces:
    return_value= list(filtered_soma_list_components),run_time,filtered_soma_list_sdf_components,glia_pieces, nuclei_pieces
else:
    return_value = list(filtered_soma_list_components),run_time,filtered_soma_list_sdf_components

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 15000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 2787 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/neuron_70104.off -o /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/neuron_70104_remove_interior.off -s /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/remove_interior_34102.mls
removed temporary input file: /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/neuron_70104.off
removed temporary output file: /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/neuron_70104_remove_interior.off
/notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/temp/remove_interior_34102.mls is being deleted....


WARNING - 2021-01-10 03:50:33,425 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,426 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,442 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,442 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,465 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,466 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,467 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,474 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,474 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,475 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,988 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:33,989 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:34,000 - base - face_normals all zero, ignoring!
WARNING - 20

WARNING - 2021-01-10 03:50:36,596 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,641 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,651 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,655 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,668 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,669 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,682 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,683 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,689 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,702 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,704 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,715 - base - face_normals all zero, ignoring!
WARNING - 2021-01-10 03:50:36,716 - base - face_normals all zero, ignoring!
WARNING - 20

There were 37 total interior meshes
Pieces satisfying nucleus requirements (700 <= x < 20000): 36
Pieces satisfying nucleus requirements (x >= 20000): 1
using precomputed glia pieces

 ---- Working on inside piece 0 ------
For glia mesh 0 there were 1894067 faces within 3000 distane
New mesh size is <trimesh.Trimesh(vertices.shape=(950346, 3), faces.shape=(1894067, 3))>


WARNING - 2021-01-10 03:50:43,900 - base - face_normals all zero, ignoring!


...

/meshAfterParty/trimesh_utils.py:1241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  



Going to run cgal segmentation with:
File: /notebooks/Auto_Proofreading/Soma_Splitting/Soma_Detection_Debug/902_mesh 
clusters:3 
smoothness:0.01






 Total time for run = 390.8562843799591
Before Filtering the number of somas found = 1
Performing Soma Mesh Backtracking to original mesh
# total split meshes = 294
viable_meshes = [0]
There were 292 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


----Working on soma-containing mesh piece 0----
current_soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(1335, 3), faces.shape=(2636, 3))>]
current_mesh = <trimesh.Trimesh(vertices.shape=(586966, 3), faces.shape=(1169519, 3))>

inside Soma subtraction
Total Time for soma mesh cancellation = 1.633
mesh_pieces_without_soma = [<trimesh.Trimesh(vertices.shape=(295720, 3), faces.shape=(589092, 3))>, <trimesh.Trimesh(vertices.shape=(90186, 3), faces.shape=(179687, 3))>, <trimesh.Trimesh(vertices.shape=(46156, 3), faces.shape=(91997, 3))>, <trimesh.Trimesh(vertices.shape=(34800, 3), faces.shape=(69305, 3))>, <trimesh.Trimesh(vertices.shape=(29415, 3), faces.shape=(58543, 3))>, <trimesh.


Largest hole before segmentation = 120399.80805154065, after = 458051.3393212965,
ratio = 3.8044191825057916, difference = 337651.53126975585


/meshAfterParty/networkx_utils.py:556: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)


In [33]:
nviz.plot_objects(meshes=filtered_soma_list_components)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [ ]:
np.inf/4

In [ ]:
len(list_of_largest_mesh_inner)

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh_inner,
                  meshes_colors="random")

In [ ]:
valid_soma_segments_width

In [ ]:
total_somas = []
if len(valid_soma_segments_width) > 0:
    print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
    somas_found_in_big_loop = True
    #get the meshes only if signfiicant length
    labels_list = classifier.labels_list

    for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
        submesh_face_list = np.where(classifier.labels_list == v)[0]
        soma_mesh = largest_mesh_path_inner_decimated_clean.submesh([submesh_face_list],append=True)
        total_somas.append(soma_mesh)

        # ---------- No longer doing the extra checks in here --------- #



        curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
        curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)
        
        print(f"\n\n {curr_side_len_check}, {curr_volume_check} ")

In [ ]:
nviz.plot_objects(total_somas[1],
                 meshes_colors="random")

In [ ]:
#1) Run th esegmentation algorithm again to segment the mesh
mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(total_somas[1],clusters=3,smoothness=0.01,verbose=True)
mesh_extra = np.array(mesh_extra)
nviz.plot_objects(meshes=mesh_extra,
                  meshes_colors="random"
                 )
#2) Filter out meshes by sizs and sdf threshold
mesh_extra_lens = np.array([len(kk.faces) for kk in mesh_extra])
filtered_meshes_idx = np.where((mesh_extra_lens >= soma_size_threshold) & (mesh_extra_lens <= soma_size_threshold_max) & (mesh_extra_sdf>soma_width_threshold))[0]
nviz.plot_objects(meshes=mesh_extra[filtered_meshes_idx])

In [ ]:
mesh_extra

In [ ]:
test_mesh = mesh_extra[filtered_meshes_idx][0]

side_length_check(test_mesh,side_length_ratio_threshold), soma_volume_check(test_mesh,volume_mulitplier)

In [ ]:
list_of_largest_mesh_inner

In [ ]:
nviz.plot_objects(list_of_largest_mesh_inner[-1],
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh)

In [ ]:
 if len(valid_soma_segments_width) > 0:
                print(f"      ------ Found {len(valid_soma_segments_width)} viable somas: {valid_soma_segments_width}")
                somas_found_in_big_loop = True
                #get the meshes only if signfiicant length
                labels_list = classifier.labels_list

                for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
                    submesh_face_list = np.where(classifier.labels_list == v)[0]
                    soma_mesh = largest_mesh_path_inner_decimated_clean.submesh([submesh_face_list],append=True)

                    # ---------- No longer doing the extra checks in here --------- #

                    
                    
                    curr_side_len_check = side_length_check(soma_mesh,side_length_ratio_threshold)
                    curr_volume_check = soma_volume_check(soma_mesh,volume_mulitplier)
                    
                    #1) Run th esegmentation algorithm again to segment the mesh
                    mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(soma_mesh,clusters=3,smoothness=0.2,verbose=True)
                    mesh_extra = np.array(mesh_extra)

                    #2) Filter out meshes by sizs and sdf threshold
                    mesh_extra_lens = np.array([len(kk.faces) for kk in mesh_extra])
                    filtered_meshes_idx = np.where((mesh_extra_lens >= soma_size_threshold) & (mesh_extra_lens <= soma_size_threshold_max) & (mesh_extra_sdf>soma_width_threshold))[0]
                    

In [ ]:
mesh_extra

In [ ]:
mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(soma_mesh,clusters=3,smoothness=0.05,verbose=True)
nviz.plot_objects(meshes=mesh_extra,
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh_inner,
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=filtered_soma_list_components,
                 meshes_colors="random")

In [ ]:
soma_mesh = original_mesh_soma(
            mesh = recov_orig_mesh_no_interior,
            soma_meshes=[soma_mesh_poisson],
            sig_th_initial_split=15)[0]

In [ ]:
for v,sdf in zip(valid_soma_segments_width,valid_soma_segments_sdf):
    submesh_face_list = np.where(classifier.labels_list == v)[0]
    soma_mesh = largest_mesh_path_inner_decimated_clean.submesh([submesh_face_list],append=True)

In [ ]:
nviz.plot_objects(soma_mesh)

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh_inner,
                  meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh)

In [ ]:
nviz.plot_objects(meshes=filtered_soma_list_components)

In [ ]:
"""
# ----------- 1/7/21 ---------------#
Now was to stitch the somas together if they are touching

"""
connected_meshes_components = tu.mesh_list_connectivity(meshes=filtered_soma_list,
                         main_mesh=recov_orig_mesh,
                                            return_connected_components=True)

filtered_soma_list_components = [tu.combine_meshes(filtered_soma_list[k]) for k in connected_meshes_components]
filtered_soma_list_sdf_components = [np.mean(filtered_soma_list_sdf[k]) for k in connected_meshes_components]


In [ ]:
filtered_soma_list_components

In [ ]:
filtered_soma_list_sdf

In [ ]:
nviz.plot_objects(meshes=filtered_soma_list,
                  meshes_colors="random")

In [ ]:
soma_mesh = original_mesh_soma(
                                            mesh = recov_orig_mesh_no_interior,
                                            soma_meshes=[soma_mesh_poisson],
                                            sig_th_initial_split=15)[0]
soma_mesh

In [ ]:
seperate_soma_meshes = su.decompress_pickle("seperate_soma_meshes")
nviz.plot_objects(meshes=seperate_soma_meshes,
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=filtered_soma_list_saved)

In [ ]:
filtered_soma_list_saved = su.decompress_pickle("filtered_soma_list")

In [ ]:
nviz.plot_objects(meshes=total_soma_list_revised)

In [ ]:
return_value

In [ ]:
nviz.plot_objects(current_neuron,
                #meshes=[return_value[0][0]],
                 meshes_colors="red")

In [ ]:
""" ((h > soma_width_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] > soma_size_threshold)
                                                                and (classifier.sdf_final_dict[g]["n_faces"] < soma_size_threshold_max))"""

In [ ]:
mesh_extra, mesh_extra_sdf = tu.mesh_segmentation(soma_mesh,clusters=3,smoothness=0.2,verbose=True)
mesh_extra = np.array(mesh_extra)


mesh_extra_lens = np.array([len(kk.faces) for kk in mesh_extra])
filtered_meshes_idx = np.where((mesh_extra_lens >= soma_size_threshold) & (mesh_extra_lens <= soma_size_threshold_max) & (mesh_extra_sdf>soma_width_threshold))[0]

if len(filtered_meshes_idx) > 0:
    filtered_meshes = mesh_extra[filtered_meshes_idx]
    filtered_meshes_sdf = mesh_extra_sdf[filtered_meshes_idx]
    
    soma_mesh_retry = filtered_meshes[np.argmax(filtered_meshes_sdf)]

nviz.plot_objects(soma_mesh_retry)

In [ ]:
soma_size_threshold

In [ ]:
nviz.plot_objects(mesh_extra[0],
                 meshes_colors="random")

In [ ]:
import trimesh_utils as tu
tu.

In [ ]:
nviz.plot_objects(largest_mesh_path_inner_decimated_clean,
                 meshes=[soma_mesh],
                 meshes_colors="red")

In [ ]:
large_mesh_threshold_inner

In [ ]:
nviz.plot_objects(list_of_largest_mesh_inner[0])

In [ ]:
list_of_largest_mesh_inner = [k for k in ordered_mesh_splits_inner if len(k.faces) > large_mesh_threshold_inner]

In [ ]:
len(list_of_largest_mesh_inner)

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh_inner,
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=[k for k in ordered_mesh_splits_inner if len(k.faces)>800],
                 meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=dec_splits,
                 meshes_colors="random")

In [ ]:
len(list_of_largest_mesh)

In [ ]:
nviz.plot_objects(list_of_largest_mesh[1],
                  meshes=[soma_mesh],
                 meshes_colors="red")

In [ ]:
list_of_largest_mesh

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh,
                 meshes_colors=["red","black"],
                 )

In [ ]:
nviz.plot_objects(meshes=list_of_largest_mesh_inner,
                 meshes_colors="random")

In [ ]:
filtered_soma_list

In [ ]:
nviz.plot_objects(meshes=filtered_soma_list)

In [ ]:
nviz.plot_objects(main_mesh=current_neuron,
                 meshes=filtered_soma_list,
                 meshes_colors="red")

In [ ]:
soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                                 current_neuron.vertices,
                                                 current_neuron.faces)